In [ ]:
# Initial creation date: April 4, 2024
# Portions of this code are adapted from:

#https://www.kaggle.com/code/lucamassaron/fine-tune-mistral-v0-2-for-sentiment-analysis
#https://ubiai.tools/fine-tuning-mistral-for-ner/

In [ ]:
pip install -q -U torch=='2.0.0'

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.4/168.4 MB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 MB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.6/102.6 MB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.2/173.2 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.1/177.1 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63

In [ ]:
pip install -q -U accelerate=='0.25.0' peft=='0.7.1' bitsandbytes=='0.41.3.post2' transformers=='4.36.1' trl=='0.7.4'

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.8/126.8 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.3/168.3 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 59.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.9/133.9 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 76.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.8/103.8 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 33.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.1/316.1 kB 22.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 36.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
from google.colab import userdata
os.environ["HF_TOKEN"] = userdata.get('HF_TOKEN')

In [ ]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import bitsandbytes as bnb
import torch
import torch.nn as nn
import transformers
from datasets import Dataset
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model
from trl import SFTTrainer
from transformers import (AutoModelForCausalLM,
                          AutoTokenizer,
                          BitsAndBytesConfig,
                          TrainingArguments,
                          pipeline,
                          logging,
                          TextStreamer)
from huggingface_hub import notebook_login
#base_model = "mistralai/Mistral-7B-v0.1"
#base_model = "BioMistral/BioMistral-7B"
#base_model = "BioMistral/BioMistral-7B-DARE"
#base_model = "epfl-llm/meditron-7b"
base_model = "meta-llama/Llama-2-7b-hf"
#base_model = "meta-llama/Llama-2-13b-hf"

# Me-LLama was downloaded from: "https://physionet.org/content/me-llama/1.0.0/MeLLaMA-13B/", then stored in our own google_drive.
#base_model = "/content/drive/MyDrive/DocDiscNER/Checkpoints/Me-LLaMa"

/usr/local/lib/python3.10/dist-packages/trl/trainer/ppo_config.py:141: UserWarning: The `optimize_cuda_cache` arguement will be deprecated soon, please use `optimize_device_cache` instead.
  warnings.warn(


Dataset

In [ ]:
import pandas as pd

Training_path = "/content/drive/MyDrive/DocDiscNER/Preprocessed_Datasets/Format2/ShaRe14/chunk_percentile95/Train-Doc-ShaRe-14.csv"
Validation_path = "/content/drive/MyDrive/DocDiscNER/Preprocessed_Datasets/Format2/ShaRe14/chunk_percentile95/Dev-Doc-ShaRe-14.csv"
Test_path = "/content/drive/MyDrive/DocDiscNER/Preprocessed_Datasets/Format2/ShaRe14/chunk_percentile95/Test-Doc-ShaRe-14.csv"

Training = pd.read_csv(Training_path, names=["Text", "Spans"], encoding="utf-8", skiprows=1)
Validation = pd.read_csv(Validation_path, names=["Text", "Spans"], encoding="utf-8",skiprows=1)
Test = pd.read_csv(Test_path, names=["Text", "Spans"], encoding="utf-8",skiprows=1)

In [ ]:
#Input representation:
def generate_prompt(data_point):
    return f"""
            Context:
            [{data_point["Text"]}]
            Spans:
            {data_point["Spans"]}
            """.strip()

def generate_test_prompt(data_point):
    return f"""
            Context:
            [{data_point["Text"]}]
            Spans:""".strip()

In [ ]:
Training = pd.DataFrame(Training.apply(generate_prompt, axis=1), columns=["Text"])
Validation = pd.DataFrame(Validation.apply(generate_prompt, axis=1), columns=["Text"])
Test = pd.DataFrame(Test.apply(generate_test_prompt, axis=1), columns=["Text"])

train_data = Dataset.from_pandas(Training)
val_data = Dataset.from_pandas(Validation)
test_data = Dataset.from_pandas(Test)

In [ ]:
print(val_data[0])

{'Text': 'Context:\n            [9436 |||| 22302 |||| 18959 |||| DISCHARGE_SUMMARY |||| 2010-01-05 00:00:00.0 |||| |||| |||| |||| Admission Date: Discharge Date: Date of Birth: Sex: F Service: Neurology HISTORY OF PRESENT ILLNESS: The patient is a 76 year old right handed woman who went to the drug store on and developed a right frontal headache. She came home and was not able to read her mail. She said that, at that time, her vision was very blurry. She could not figure out how to use her television or other appliances, including her telephone. There was no trauma or loss of consciousness. The patient called her daughter at that time and went to an outside hospital, where she had a CT scan showing blood in the left occipital lobe and extension into the ventricle. The patient had no nausea or vomiting. She was transferred to and admitted to the Intensive Care Unit overnight, without any complications. The following day, the patient was transferred to the floor.]\n            Spans:\n  

Model

In [ ]:
# Load base model(Mistral 2B)
bnb_config = BitsAndBytesConfig(
    load_in_4bit= True,
    bnb_4bit_quant_type= "nf4",
    bnb_4bit_compute_dtype= torch.bfloat16,
    bnb_4bit_use_double_quant= False,
)
model = AutoModelForCausalLM.from_pretrained(
   base_model,
    quantization_config=bnb_config,
    device_map={"": 0}
)
model.config.use_cache = False
model.config.pretraining_tp = 1
model.gradient_checkpointing_enable()

# tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model, token=os.environ['HF_TOKEN'])
tokenizer.padding_side = 'left' # I used 'right' for all models, except for 'LLaMA2-7B'
tokenizer.pad_token = tokenizer.unk_token
tokenizer.add_eos_token = True
model.config.pad_token_id = tokenizer.pad_token_id


#Adding the adapters in the layers
model = prepare_model_for_kbit_training(model)
peft_config = LoraConfig(
        r=256,
        lora_alpha=128,
        lora_dropout=0.1, # Tested 0.05, but 0.1 yielded better results
        bias="none",
        task_type="CAUSAL_LM",
        target_modules=["q_proj", "k_proj", "v_proj", "o_proj","gate_proj"] # attempted adding "up_proj", "down_proj", "lm_head", but saw no improvement.
    )
model = get_peft_model(model, peft_config)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/609 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/776 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [ ]:
#prints the number of trainable parameters in the model
trainable_params = 0
all_param = 0
for _, param in model.named_parameters():
    all_param += param.numel()
    if param.requires_grad:
        trainable_params += param.numel()
print(f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}")

trainable params: 369098752 || all params: 4909699072 || trainable%: 7.51774694512275


In [ ]:
training_arguments = TrainingArguments(
    output_dir="logs",
    num_train_epochs=20,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    gradient_accumulation_steps=8, # 4
    optim="paged_adamw_32bit",
    save_steps=0,
    logging_steps=25,
    learning_rate=5e-4,
    weight_decay=0.001,
    fp16=True,
    bf16=False,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="linear", #Tested "cosine", but "linear" yielded better results"
    report_to="tensorboard",
    evaluation_strategy="epoch"
)

In [ ]:
trainer = SFTTrainer(
    model=model,
    train_dataset=train_data,
    eval_dataset=val_data,
    peft_config=peft_config,
    dataset_text_field="Text",
    tokenizer=tokenizer,
    args=training_arguments,
    packing=False,
    max_seq_length=1024,
)

Map:   0%|          | 0/1016 [00:00<?, ? examples/s]

Map:   0%|          | 0/813 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:247: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(


In [ ]:
# monitering using wandb
wandb.login(key = "--")
run = wandb.init(project='DocDiscNER', job_type="training", anonymous="allow")

In [ ]:
import torch
torch.cuda.empty_cache()
trainer.train()

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss
0,No log,1.369169
1,1.394100,1.268476
2,1.394100,1.278310
3,0.912500,1.322711
4,0.622300,1.413449
5,0.622300,1.542055
6,0.408000,1.639822
8,0.245100,1.817299
9,0.153100,1.916073
10,0.153100,2.014628


TrainOutput(global_step=300, training_loss=0.34072917997837066, metrics={'train_runtime': 6333.0552, 'train_samples_per_second': 3.209, 'train_steps_per_second': 0.047, 'total_flos': 3.706825694842061e+17, 'train_loss': 0.34072917997837066, 'epoch': 18.9})

In [ ]:
trainer.save_model("/content/drive/MyDrive/DocDiscNER/Checkpoints/ShaRe14/LLaMA2-7B")

In [ ]:
model_path = "/content/drive/MyDrive/DocDiscNER/Checkpoints/ShaRe14/LLaMA2-7B"

In [ ]:
model = AutoModelForCausalLM.from_pretrained(model_path,
      quantization_config=bnb_config,
      device_map={"": 0})

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
wandb.finish()

In [ ]:
%load_ext tensorboard
%tensorboard --logdir logs/runs

In [ ]:
############### PREDICT ###############

In [ ]:
####################Greedy Decoding###########
# skip the following four code blocks to decode using SCD with beam search.

In [ ]:
def predict(X_test, model, tokenizer):
    prompt = X_test
    pipe = pipeline(task="text-generation",
                    model=model,
                    tokenizer=tokenizer,
                    max_new_tokens = 300,
                    temperature = 0,
                    )
    result = pipe(prompt, pad_token_id=pipe.tokenizer.eos_token_id)
    answer = result[0]['generated_text'].split("Spans:")[-1].strip()

    return answer

In [ ]:
# Trying one example
output = predict(test_data[0]["Text"], model, tokenizer)
print(output)

disorder: frontal headache; disorder: vision blurry; disorder: trauma; disorder: loss of consciousness; disorder: blood in lobe; disorder: blood in ventricle; disorder: nausea; disorder: vomiting


In [ ]:
# FOR GREEDY : Iterate over the All test examples
outputs = []

#for data in test_data:
num_examples = len(test_data)
for i in range(num_examples):
    print(i)
    output = predict(test_data[i]["Text"], model, tokenizer)
    print(output)
    outputs.append(output)

0
disorder: frontal headache; disorder: vision blurry; disorder: trauma; disorder: loss of consciousness; disorder: blood in lobe; disorder: blood in ventricle; disorder: nausea; disorder: vomiting
1
disorder: Memory problems; disorder: Histoplasmosis
2
disorder: Abdomen nondistended; disorder: Abdomen nontender; disorder: Abdomen Soft; disorder: murmur; disorder: paraphasic errors; disorder: difficulty putting them together; disorder: difficulty sentences; disorder: hemifield defect; disorder: visual fields defect; disorder: face symmetric; disorder: sensation intact; disorder: tongue midline; disorder: sensory extinction; disorder: decreased reflexes
3
disorder: bleed; disorder: hydrocephalus; disorder: bleed; disorder: bleed; disorder: neoplastic disease; disorder: cyst; disorder: ovarian cyst
4
disorder: alexia; disorder: bleed; disorder: amyloid angiopathy
5
disorder: Esophageal carcinoma; disorder: adenocarcinoma; disorder: esophagus; disorder: symptomatic; disorder: weight loss;

In [ ]:
# FOR GREEDY: write the prediction in file
import csv
Txt = []
Tgt = []

with open("/content/drive/MyDrive/DocDiscNER/Preprocessed_Datasets/Format2/ShaRe14/chunk_percentile95/Test-Doc-ShaRe-14.csv", 'r', encoding='utf-8') as csv_file:
        reader = csv.reader(csv_file)
        next(reader)
        for row in reader:
            Txt.append(row[0])
            Tgt.append(row[1])
data = list(zip(Txt, outputs, Tgt))
file_path = '/content/drive/MyDrive/DocDiscNER/Results/ShaRe14/LLaMA2-7B.tsv'

with open(file_path, "w", newline="", encoding="utf-8") as tsvfile:
    writer = csv.writer(tsvfile, delimiter="\t")
    writer.writerow(["Text", "Output", "Target"])  #   header row
    writer.writerows(data)  # data rows

In [ ]:
#################### Beam Search Decoding ###############
# we will return 4 beams here. After saving the resulting TSV file, proceed to the notebook: DocDiscNER_SelfConsistency.ipynb

In [ ]:
#FOR BEAM SEARCH: saving all the beams in a list
def predict(X_test, model, tokenizer):
    prompt = X_test
    pipe = pipeline(task="text-generation",
                    model=model,
                    tokenizer=tokenizer,
                    max_new_tokens = 300,
                    temperature = 0.0,
                    num_beams=4, num_return_sequences=4
                    )
    result = pipe(prompt, pad_token_id=pipe.tokenizer.eos_token_id)
    Allbeams = []
    for i in range(len(result)):
          resul = result[i]['generated_text'].split("Spans:")[-1].strip()
          Allbeams.append(resul)
          print(resul)
    return Allbeams

In [ ]:
# Trying one example
output = predict(test_data[2]["Text"], model, tokenizer)
print(output)

The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FuyuForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MistralForCausalLM', 'MixtralForCausalLM', 'MptForCausalLM', 'MusicgenForCausalLM', 'MvpForCausalLM', 'OpenLlamaForCausalLM', 'OpenAIGPTLMHeadModel', 'OPTForCausalLM', 'PegasusForCausalLM', 'PersimmonF

disorder: end-stage renal disease; disorder: glomerulonephritis; disorder: IgA; disorder: anuric; disorder: calciphylaxis; disorder: hypercalcemia; disorder: arteriovenous fistula; disorder: injured
disorder: end-stage renal disease; disorder: glomerulonephritis; disorder: IgA; disorder: calciphylaxis; disorder: hypercalcemia; disorder: injured
disorder: end-stage renal disease; disorder: glomerulonephritis; disorder: IgA; disorder: anuric; disorder: calciphylaxis; disorder: hypercalcemia; disorder: injured
disorder: end-stage renal disease; disorder: glomerulonephritis; disorder: IgA; disorder: calciphylaxis; disorder: hypercalcemia; disorder: arteriovenous fistula; disorder: injured
disorder: end-stage renal disease; disorder: glomerulonephritis; disorder: IgA; disorder: anuric; disorder: calciphylaxis; disorder: hypercalcemia; disorder: arteriovenous fistula; disorder: injured


In [ ]:
# FOR BEAM SEARCH : Iterate over ALL BEAMS AND All test examples
beam1 = []
beam2 = []
beam3 = []
beam4 = []
#for data in test_data:
num_examples = len(test_data)
for i in range(num_examples):
    print(i)
    output = predict(test_data[i]["Text"], model, tokenizer)
    #output = generate_response(data["Text"], model)
    for j, beam in enumerate(output, 1):
        if j == 1:
            beam1.append(beam)
        elif j == 2:
            beam2.append(beam)
        elif j == 3:
            beam3.append(beam)
        elif j == 4:
            beam4.append(beam)

0


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:389: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


disorder: frontal headache; disorder: blurry; disorder: trauma; disorder: loss of consciousness; disorder: blood; disorder: nausea; disorder: vomiting
disorder: frontal headache; disorder: blurry; disorder: trauma; disorder: loss of consciousness; disorder: blood; disorder: nausea; disorder: vomiting; disorder: complications
disorder: right frontal headache; disorder: blurry; disorder: trauma; disorder: loss of consciousness; disorder: blood; disorder: nausea; disorder: vomiting
disorder: right frontal headache; disorder: blurry; disorder: trauma; disorder: loss of consciousness; disorder: blood; disorder: nausea; disorder: vomiting; disorder: complications
1
disorder: Memory problems; disorder: Histoplasmosis; disorder: cancer
disorder: Memory problems; disorder: knee replacement; disorder: Histoplasmosis; disorder: cancer
disorder: Memory problems; disorder: cancer; disorder: Histoplasmosis
disorder: Memory problems; disorder: histoplasmosis; disorder: cancer
2
disorder: hemifield de

In [ ]:
# FOR BEAM SEARCH:  write in file
import csv
Txt = []
Tgt = []
# Open a new TSV file for writing

with open("/content/drive/MyDrive/DocDiscNER/Preprocessed_Datasets/Format3/ShaRe14/Test-Doc-ShaRe-14.csv", 'r', encoding='utf-8') as csv_file:
        reader = csv.reader(csv_file)
        next(reader)
        for row in reader:
            Txt.append(row[0])
            Tgt.append(row[1])
data = list(zip(Txt, beam1, beam2, beam3, beam4, Tgt))
file_path = '/content/drive/MyDrive/DocDiscNER/Results/ShaRe14/BioMistBeam.tsv'

with open(file_path, "w", newline="", encoding="utf-8") as tsvfile:
    writer = csv.writer(tsvfile, delimiter="\t")
    writer.writerow(["Text", "Beam1", "Beam2", "Beam3", "Beam4", "Target"])  # Write the header row
    writer.writerows(data)  # Write the data rows

In [ ]:
# next, use the resulting TSV file in the notebook: DocDiscNER_SelfConsistency.ipynb


In [ ]:
from google.colab import runtime
runtime.unassign()